In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import roc_auc_score
#from tqdm.notebook import tqdm_notebook

from util import load_data, load_data_fashion, create_dataset

import time

C:\Users\An-ck\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\An-ck\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\An-ck\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\An-ck\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

In [2]:
(x_train, y_train), (x_test, y_test) = load_data(reshape=2)

In [3]:
def create_model():
    # CONV AE
    model = keras.Sequential([
        layers.Input( shape=(28, 28, 1) ),
        # Reduce ->
        layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Reduce ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Reduce -> 
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Increase ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.UpSampling2D((2,2)),
        # Increase ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.UpSampling2D((2,2)),
        # Increase ->
        layers.Conv2D(16, (3,3), activation='relu'),
        layers.UpSampling2D((2,2)),
        # Final
        layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=1, caseii=False, caseiii=False):
    callbacks = [
        keras.callbacks.EarlyStopping(patience=2)
    ]
    if caseii:
        train = x_train[y_train!=normal]
        test  = x_test[y_test!=normal]
    elif caseiii:
        train = x_train
        test  = x_test
    else:
        train = x_train[y_train==normal]
        test  = x_test[y_test==normal]
    history = model.fit(train, train,
                    epochs=50,
                    batch_size=256,
                    verbose=verbose,
                    shuffle=True,
                    validation_data=(test, test),
                    callbacks=callbacks)
    return model

In [ ]:
model = create_model()
start = time.process_time()

model = fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=0)

end = time.process_time()
print(end - start)

start = time.process_time()
xhat = model.predict(x_test)
end = time.process_time()
print(end - start)

In [ ]:
evals = np.zeros((10, 30))
for i in range(10):
    # Evaluate for all numbers
    for j in tqdm_notebook(range(5)):
        # Evaluate each method 30 times
        model = create_model()
        model = fit_model(model, x_train, y_train, x_test, y_test, normal=i)

        x = np.copy( x_test )
        y = y_test
        labels = np.copy( y )
        labels[ y == i ] = 0
        labels[ y != i ] = 1
        
        xhat = model.predict(x)
        
        x = x.reshape(len(x), 28*28)
        xhat = xhat.reshape(len(xhat), 28*28)
        
        err  = np.sum(np.abs(x-xhat), axis=1)
        # Max-Min normalize the error
        err /= np.max(err)
        # Compute AUC
        AUC = roc_auc_score(labels, err)
        evals[i,j] = AUC
    print(np.mean(evals[i,:])*100)

print(np.mean(evals[:,:5], axis=1))
print(np.std(evals, axis=1))

In [ ]:
print(np.mean(evals[:,:5], axis=1))
print(np.std(evals[:,:5], axis=1))

# CASE 2

In [5]:
evals = np.zeros((10))
for i in range(10):
    # Evaluate each method 30 times
    model = create_model()
    model = fit_model(model, x_train, y_train, x_test, y_test, normal=i, caseii=True)

    x = np.copy( x_test )
    y = y_test
    labels = np.copy( y )
    labels[ y != i ] = 0
    labels[ y == i ] = 1

    xhat = model.predict(x)

    x = x.reshape(len(x), 28*28)
    xhat = xhat.reshape(len(xhat), 28*28)

    err  = np.sum(np.abs(x-xhat), axis=1)
    # Max-Min normalize the error
    err /= np.max(err)
    # Compute AUC
    AUC = roc_auc_score(labels, err)
    evals[i] = AUC

print(evals)

Epoch 1/50
212/212 [==============================] - 33s 158ms/step - loss: 0.2800 - val_loss: 0.1703
Epoch 2/50
212/212 [==============================] - 31s 146ms/step - loss: 0.1539 - val_loss: 0.1410
Epoch 3/50
212/212 [==============================] - 33s 155ms/step - loss: 0.1353 - val_loss: 0.1293
Epoch 4/50
212/212 [==============================] - 35s 163ms/step - loss: 0.1268 - val_loss: 0.1228
Epoch 5/50
212/212 [==============================] - 31s 146ms/step - loss: 0.1215 - val_loss: 0.1182
Epoch 6/50
212/212 [==============================] - 34s 161ms/step - loss: 0.1175 - val_loss: 0.1162
Epoch 7/50
212/212 [==============================] - 35s 163ms/step - loss: 0.1146 - val_loss: 0.1119
Epoch 8/50
212/212 [==============================] - 35s 163ms/step - loss: 0.1121 - val_loss: 0.1099
Epoch 9/50
212/212 [==============================] - 38s 177ms/step - loss: 0.1099 - val_loss: 0.1078
Epoch 10/50
212/212 [==============================] - 36s 168ms/step - l

# CASE 3

In [4]:
(f_x_train, f_y_train), (f_x_test, f_y_test) = load_data_fashion(reshape=2)

# Evaluate each method 30 times
model = create_model()
model = fit_model(model, x_train, y_train, x_test, y_test, verbose=1, caseiii=True)

x = np.copy( x_test )
x[5000:] = f_x_test[:5000]

labels = np.zeros( y_test.shape )
labels[5000:] = 1

xhat = model.predict(x)

x = x.reshape(len(x), 28*28)
xhat = xhat.reshape(len(xhat), 28*28)

err  = np.sum(np.abs(x-xhat), axis=1)
# Max-Min normalize the error
err /= np.max(err)
# Compute AUC
AUC = roc_auc_score(labels, err)
print(AUC)

Epoch 1/50
235/235 [==============================] - 31s 130ms/step - loss: 0.2759 - val_loss: 0.1813
Epoch 2/50
235/235 [==============================] - 30s 127ms/step - loss: 0.1629 - val_loss: 0.1493
Epoch 3/50
235/235 [==============================] - 34s 145ms/step - loss: 0.1421 - val_loss: 0.1345
Epoch 4/50
235/235 [==============================] - 38s 162ms/step - loss: 0.1308 - val_loss: 0.1257
Epoch 5/50
235/235 [==============================] - 38s 160ms/step - loss: 0.1240 - val_loss: 0.1206
Epoch 6/50
235/235 [==============================] - 35s 150ms/step - loss: 0.1197 - val_loss: 0.1170
Epoch 7/50
235/235 [==============================] - 37s 156ms/step - loss: 0.1167 - val_loss: 0.1143
Epoch 8/50
235/235 [==============================] - 33s 139ms/step - loss: 0.1145 - val_loss: 0.1124
Epoch 9/50
235/235 [==============================] - 31s 132ms/step - loss: 0.1127 - val_loss: 0.1107
Epoch 10/50
235/235 [==============================] - 33s 140ms/step - l